# ONNX教程 - 第1部分：引言与基础准备

欢迎来到ONNX教程系列的第一部分！在本notebook中，我们将介绍ONNX的基本概念，并设置必要的开发环境。

## 什么是ONNX？

ONNX (Open Neural Network Exchange) 是一种开放格式，用于表示深度学习模型。ONNX定义了一组公共操作符和通用数据类型，使AI开发人员能够使用各种框架（如PyTorch、TensorFlow、MXNet等）构建模型，然后将这些模型导出为ONNX格式，以便在不同平台和设备上运行。

### ONNX的主要优势：

1. **跨框架互操作性**：在不同深度学习框架之间迁移模型
2. **硬件加速**：利用针对不同硬件优化的推理引擎（如ONNX Runtime）
3. **部署灵活性**：在云端、边缘设备或移动设备上部署模型
4. **模型优化**：通过ONNX工具进行模型压缩和优化

### 教程概述

本教程系列包含以下部分：

1. 引言与基础准备（本部分）
2. 创建和训练PyTorch模型
3. 将PyTorch模型导出为ONNX格式
4. ONNX模型的基本操作

让我们开始检查环境并演示一个简单的PyTorch模型！

In [ ]:
# 导入必要的库
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

## 1. 环境检查

首先，我们需要检查已安装的库版本并确认ONNX及相关工具是否可用。

In [ ]:
def check_environment():
    """检查环境配置"""
    print("Python版本:", sys.version)
    print("PyTorch版本:", torch.__version__)
    
    # 检查ONNX是否已安装
    try:
        import onnx
        print("ONNX版本:", onnx.__version__)
        onnx_available = True
    except ImportError:
        print("ONNX未安装，请使用pip install onnx安装")
        onnx_available = False
    
    # 检查ONNX Runtime是否已安装  
    try:
        import onnxruntime
        print("ONNX Runtime版本:", onnxruntime.__version__)
        onnxruntime_available = True
    except ImportError:
        print("ONNX Runtime未安装，请使用pip install onnxruntime安装")
        onnxruntime_available = False
    
    # 检查CUDA是否可用
    print("CUDA是否可用:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("CUDA版本:", torch.version.cuda)
        print("当前CUDA设备:", torch.cuda.current_device())
        print("CUDA设备名称:", torch.cuda.get_device_name(0))
        
    # 检查可视化工具Netron
    try:
        import netron
        print("Netron版本: 可用")
        netron_available = True
    except ImportError:
        print("Netron未安装，请使用pip install netron安装 (用于可视化ONNX模型)")
        netron_available = False
    
    return onnx_available and onnxruntime_available

# 执行环境检查
print("[1] 环境检查")
env_ready = check_environment()

## 2. 安装缺失的包

如果某些必要的包未安装，您可以使用以下命令进行安装：

In [ ]:
# 如果需要安装缺失的包，取消以下注释
# !pip install onnx onnxruntime netron

## 3. 创建简单的PyTorch模型

接下来，我们将定义一个简单的全连接神经网络作为示例：

In [ ]:
class SimpleModel(nn.Module):
    """一个简单的全连接神经网络"""
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(10, 20)
        self.fc2 = nn.Linear(20, 5)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 实例化模型
model = SimpleModel()
print("\n模型结构:")
print(model)

## 4. 测试模型的前向传播

现在让我们创建一个随机输入并测试模型的前向传播：

In [ ]:
def test_simple_model(model):
    """测试简单模型的前向传播"""
    model.eval()  # 设置为评估模式
    
    # 创建一个随机输入
    dummy_input = torch.randn(1, 10)
    print("\n输入形状:", dummy_input.shape)
    print("输入数据:\n", dummy_input)
    
    # 前向传播
    with torch.no_grad():
        output = model(dummy_input)
    
    print("\n输出形状:", output.shape)
    print("输出数据:\n", output)
    
    return dummy_input

# 执行模型测试
print("[2] 简单PyTorch模型测试")
dummy_input = test_simple_model(model)

## 5. ONNX格式简介

ONNX模型由以下关键组件组成：

1. **模型元数据**：包含版本信息、生产者信息等
2. **图 (Graph)**：模型的计算图表示
3. **节点 (Node)**：图中的操作（如卷积、ReLU等）
4. **张量 (Tensor)**：数据和参数
5. **属性 (Attribute)**：节点的参数配置

下面我们将展示ONNX模型的结构示意图：

In [ ]:
# 使用ASCII来表示ONNX模型的结构
onnx_structure = """
+------------------------+
| ONNX Model             |
+------------------------+
| - IR Version           |
| - Opset Version        |
| - Producer Name        |
| - Model Version        |
+----------+-------------+
|          |
|          v
+----------+-------------+
| Graph                  |
+------------------------+
| - Input Tensors        |
| - Output Tensors       |
| - Initializers (Weights)|
+----------+-------------+
|          |
|          v
+----------+-------------+
| Nodes (Operations)     |
+------------------------+
| - Operators (Conv, ReLU)|
| - Attributes           |
| - Input/Output Edges   |
+------------------------+
"""

print(onnx_structure)

## 6. 预览PyTorch转ONNX流程

在下一个教程部分中，我们将详细介绍如何将PyTorch模型导出为ONNX格式，但这里先简要预览一下流程的代码结构：

In [ ]:
# 预览代码 - 不执行
def preview_export_code():
    print("""# PyTorch模型转换为ONNX的基本步骤
import torch
import onnx

# 1. 准备模型和输入
model = SimpleModel()  # 定义模型
dummy_input = torch.randn(1, 10)  # 创建样本输入

# 2. 导出为ONNX格式
torch.onnx.export(model,               # 模型
                  dummy_input,         # 模型输入
                  "simple_model.onnx", # 输出文件名
                  export_params=True,  # 存储训练参数权重
                  opset_version=12,    # ONNX版本
                  input_names=["input"],    # 输入名
                  output_names=["output"],  # 输出名
                  dynamic_axes={"input": {0: "batch_size"},
                               "output": {0: "batch_size"}})

# 3. 验证ONNX模型
onnx_model = onnx.load("simple_model.onnx")
onnx.checker.check_model(onnx_model)""")

# 显示预览代码
preview_export_code()

## 7. ONNX生态系统

ONNX生态系统包含许多工具和库，下面是一些常用的组件：

In [ ]:
onnx_ecosystem = {
    "核心库": [
        "ONNX: 定义模型格式和操作符",
        "ONNX Runtime: 高性能推理引擎"
    ],
    "转换工具": [
        "PyTorch: torch.onnx.export()",
        "TensorFlow: tf2onnx",
        "Keras: keras2onnx"
    ],
    "优化工具": [
        "ONNX Runtime: 图优化",
        "ONNX Simplifier: 模型简化",
        "TensorRT: NVIDIA GPU优化"
    ],
    "可视化工具": [
        "Netron: 图形化查看ONNX模型"
    ],
    "部署平台": [
        "云端: Azure ML, AWS SageMaker",
        "边缘设备: NVIDIA Jetson, Intel NCS",
        "移动设备: Android NNAPI, iOS CoreML"
    ]
}

# 打印ONNX生态系统信息
print("ONNX生态系统组件:")
for category, tools in onnx_ecosystem.items():
    print(f"\n{category}:")
    for tool in tools:
        print(f"  - {tool}")

## 8. 总结

在本教程中，我们：

1. 介绍了ONNX的概念和优势
2. 检查了开发环境和必要的库
3. 创建并测试了一个简单的PyTorch模型
4. 了解了ONNX模型的基本结构
5. 预览了PyTorch模型导出为ONNX的流程
6. 了解了ONNX生态系统的主要组件

在下一个教程中，我们将创建和训练一个更实用的PyTorch模型（MNIST手写数字识别），为后续的ONNX转换做准备。

In [ ]:
print("\n环境检查完成。")
if env_ready:
    print("所有必要的库都已正确安装，您可以继续下一部分的学习！")
else:
    print("请安装缺少的库后继续。")